<a href="https://colab.research.google.com/github/CV5S/damagetable_SW2.5/blob/main/Critsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 威力計算の仕組み
## 固定値
なんらかの方法で固定的なダメージが入る。
これも実は勘定に入れる必要がある。自動失敗のアレがあるからね。

## 威力表
2dを振り、威力表に基づいて打点の計算が行われる。出目に応じては特別な効果もある。
- 2: 自動失敗 (威力0)
- 2~C値未満: 普通にダメージを加算
- C値以上: 威力表をもう一周
2は実質的に打点が0なだけと考えてもよいだろう。よいのか？1週目で2を引いた場合は固定値も反映されなくなる、それを勘定に入れる必要がある。

### 特殊処理
威力表の振り足しには干渉する方法がいくつかある。

#### 必殺攻撃
威力表の出目が3~11のときのみ、それを+1する。つまり1ゾロは誤魔化せないし、12を超えることもない。

#### クリティカルレイ
一周目だけ威力表の出目に加算する。効果はB/A/S/SSでそれぞれ+1/+2/+3/+6。
もちろん13以上にはならないが、1ゾロは絶対に出ない。

#### 首切り刀
振り足しが発生するごとに威力を+5する。最大でも100までにしかならないという性質はある。

In [116]:
# インポート
import random
import pandas as pd
from tqdm import tqdm
import statistics

In [117]:
# 威力表配列
damage_table = [
                #0~10
  [0, 0, 0, 1, 2, 2, 3, 3, 4, 4],
  [0, 0, 0, 1, 2, 3, 3, 3, 4, 4],
  [0, 0, 0, 1, 2, 3, 4, 4, 4, 4],
  [0, 0, 1, 1, 2, 3, 4, 4, 4, 5],
  [0, 0, 1, 2, 2, 3, 4, 4, 5, 5],
  [0, 1, 1, 2, 2, 3, 4, 5, 5, 5],
  [0, 1, 1, 2, 3, 3, 4, 5, 5, 5],
  [0, 1, 1, 2, 3, 4, 4, 5, 5, 6],
  [0, 1, 2, 2, 3, 4, 4, 5, 6, 6],
  [0, 1, 2, 3, 3, 4, 4, 5, 6, 7],
  [1, 1, 2, 3, 3, 4, 5, 5, 6, 7],

                 #11~20
  [1, 2, 2, 3, 3, 4, 5, 6, 6, 7],
  [1, 2, 2, 3, 4, 4, 5, 6, 6, 7],
  [1, 2, 3, 3, 4, 4, 5, 6, 7, 7],
  [1, 2, 3, 4, 4, 4, 5, 6, 7, 8],
  [1, 2, 3, 4, 4, 5, 5, 6, 7, 8],
  [1, 2, 3, 4, 4, 5, 6, 7, 7, 8],
  [1, 2, 3, 4, 5, 5, 6, 7, 7, 8],
  [1, 2, 3, 4, 5, 6, 6, 7, 7, 8],
  [1, 2, 3, 4, 5, 6, 7, 7, 8, 9],
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],

                 #21~30
  [1, 2, 3, 4, 6, 6, 7, 8, 9, 10],
  [1, 2, 3, 5, 6, 6, 7, 8, 9, 10],
  [2, 2, 3, 5, 6, 7, 7, 8, 9, 10],
  [2, 3, 4, 5, 6, 7, 7, 8, 9, 10],
  [2, 3, 4, 5, 6, 7, 8, 8, 9, 10],
  [2, 3, 4, 5, 6, 8, 8, 9, 9, 10],
  [2, 3, 4, 6, 6, 8, 8, 9, 9, 10],
  [2, 3, 4, 6, 6, 8, 9, 9, 10, 10],
  [2, 3, 4, 6, 7, 8, 9, 9, 10, 10],
  [2, 4, 4, 6, 7, 8, 9, 10, 10, 10],

                 #31~40
  [2, 4, 5, 6, 7, 8, 9, 10, 10, 11],
  [3, 4, 5, 6, 7, 8, 10, 10, 10, 11],
  [3, 4, 5, 6, 8, 8, 10, 10, 10, 11],
  [3, 4, 5, 6, 8, 9, 10, 10, 11, 11],
  [3, 4, 5, 7, 8, 9, 10, 10, 11, 12],
  [3, 5, 5, 7, 8, 9, 10, 11, 11, 12],
  [3, 5, 6, 7, 8, 9, 10, 11, 12, 12],
  [3, 5, 6, 7, 8, 10, 10, 11, 12, 13],
  [4, 5, 6, 7, 8, 10, 11, 11, 12, 13],
  [4, 5, 6, 7, 9, 10, 11, 11, 12, 13],

                 #41~50
  [4, 6, 6, 7, 9, 10, 11, 12, 12, 13],
  [4, 6, 7, 7, 9, 10, 11, 12, 13, 13],
  [4, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  [4, 6, 7, 8, 10, 10, 11, 12, 13, 14],
  [4, 6, 7, 9, 10, 10, 11, 12, 13, 14],
  [4, 6, 7, 9, 10, 10, 12, 13, 13, 14],
  [4, 6, 7, 9, 10, 11, 12, 13, 13, 15],
  [4, 6, 7, 9, 10, 12, 12, 13, 13, 15],
  [4, 6, 7, 10, 10, 12, 12, 13, 14, 15],
  [4, 6, 8, 10, 10, 12, 12, 13, 15, 15],

                 #51~60
  [5, 7, 8, 10, 10, 12, 12, 13, 15, 15],
  [5, 7, 8, 10, 11, 12, 12, 13, 15, 15],
  [5, 7, 9, 10, 11, 12, 12, 14, 15, 15],
  [5, 7, 9, 10, 11, 12, 13, 14, 15, 16],
  [5, 7, 10, 10, 11, 12, 13, 14, 16, 16],
  [5, 8, 10, 10, 11, 12, 13, 15, 16, 16],
  [5, 8, 10, 11, 11, 12, 13, 15, 16, 17],
  [5, 8, 10, 11, 12, 12, 13, 15, 16, 17],
  [5, 9, 10, 11, 12, 12, 14, 15, 16, 17],
  [5, 9, 10, 11, 12, 13, 14, 15, 16, 18],

                #61~70
  [5, 9, 10, 11, 12, 13, 14, 16, 17, 18],
  [5, 9, 10, 11, 13, 13, 14, 16, 17, 18],
  [5, 9, 10, 11, 13, 13, 15, 17, 17, 18],
  [5, 9, 10, 11, 13, 14, 15, 17, 17, 18],
  [5, 9, 10, 12, 13, 14, 15, 17, 18, 18],
  [5, 9, 10, 12, 13, 15, 15, 17, 18, 19],
  [5, 9, 10, 12, 13, 15, 16, 17, 19, 19],
  [5, 9, 10, 12, 14, 15, 16, 17, 19, 19],
  [5, 9, 10, 12, 14, 16, 16, 17, 19, 19],
  [5, 9, 10, 12, 14, 16, 17, 18, 19, 19],

                #71~80
  [5, 9, 10, 13, 14, 16, 17, 18, 19, 20],
  [5, 9, 10, 13, 15, 16, 17, 18, 19, 20],
  [5, 9, 10, 13, 15, 16, 17, 19, 20, 21],
  [6, 9, 10, 13, 15, 16, 18, 19, 20, 21],
  [6, 9, 10, 13, 16, 16, 18, 19, 20, 21],
  [6, 9, 10, 13, 16, 17, 18, 19, 20, 21],
  [6, 9, 10, 13, 16, 17, 18, 20, 21, 22],
  [6, 9, 10, 13, 16, 17, 19, 20, 22, 23],
  [6, 9, 10, 13, 16, 18, 19, 20, 22, 23],
  [6, 9, 10, 13, 16, 18, 20, 21, 22, 23],

                #81~90
  [6, 9, 10, 13, 17, 18, 20, 21, 22, 23],
  [6, 9, 10, 14, 17, 18, 20, 21, 22, 24],
  [6, 9, 11, 14, 17, 18, 20, 21, 23, 24],
  [6, 9, 11, 14, 17, 19, 20, 21, 23, 24],
  [6, 9, 11, 14, 17, 19, 21, 22, 23, 24],
  [7, 10, 11, 14, 17, 19, 21, 22, 23, 25],
  [7, 10, 12, 14, 17, 19, 21, 22, 24, 25],
  [7, 10, 12, 14, 18, 19, 21, 22, 24, 25],
  [7, 10, 12, 15, 18, 19, 21, 22, 24, 26],
  [7, 10, 12, 15, 18, 19, 21, 23, 25, 26],

                #91~100
  [7, 11, 13, 15, 18, 19, 21, 23, 25, 26],
  [7, 11, 13, 15, 18, 20, 21, 23, 25, 27],
  [8, 11, 13, 15, 18, 20, 22, 23, 25, 27],
  [8, 11, 13, 16, 18, 20, 22, 23, 25, 28],
  [8, 11, 14, 16, 18, 20, 22, 23, 26, 28],
  [8, 11, 14, 16, 19, 20, 22, 23, 26, 28],
  [8, 12, 14, 16, 19, 20, 22, 24, 26, 28],
  [8, 12, 15, 16, 19, 20, 22, 24, 27, 28],
  [8, 12, 15, 17, 19, 20, 22, 24, 27, 29],
  [8, 12, 15, 18, 19, 20, 22, 24, 27, 30]
]

In [118]:
# 威力計算
## 2d
def roll_2d6():
    d1 = random.randint(1, 6)
    d2 = random.randint(1, 6)
    return d1 + d2

## 威力表
def get_damage(K, dices):
  K = min(100, K)
  damage = damage_table[K][dices-3]
  return damage

## クリティカルシミュレータ
def CritSim(K, C, bonus, armor, critbonus, hissatsu, kubikiri):
  sum = 0
  dd = min(roll_2d6()+critbonus, 12)

  if hissatsu == 1 and dd != 2 and dd != 12:
    dd += 1

  if dd == 2:
    return sum
  elif dd < C:
    sum += get_damage(K, dd)
  else:
    sum += get_damage(K, dd)
    sum += CritSim(K+5*kubikiri, C, 0, 0, 0, hissatsu, kubikiri)

  sum += bonus
  sum -= armor
  sum = max(0, sum)

  return sum

## ランダムにたくさんの試行
def Statsmaker(K, C, bonus, armor, critbonus, hissatsu, kubikiri, num_trials):
  results = []

  for i in tqdm(range(num_trials)):
    result = CritSim(K, C, bonus, armor, critbonus, hissatsu, kubikiri)
    results.append(result)

  return results


In [128]:
# パラメータの入力


# データをまとめて表示


# 複数の計算を比較できるモードも



100%|██████████| 10000/10000 [00:00<00:00, 149660.27it/s]

19.2086
